In [1]:
import requests
import time
import random
import platform
import os
import json
import pandas as pd
import numpy as np
import itertools
import pickle
from tqdm import tqdm_notebook as tqdm
from lxml import etree
from selenium import webdriver

In [41]:
class CrawlDoubanTV():
    
    user_agent = [
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60',
        'Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50',
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0',
        'Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
        'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 Safari/534.16',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.101 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)',
        'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)',
    ]
    
    def __init__(self, scrolls=25, headless=True, sleep=0.2, discover=None, inter_sleep=0):
        self.scrolls = scrolls
        self.options = webdriver.ChromeOptions()
        self.ids = []
        self.sleep = sleep
        self.inter_sleep = inter_sleep
        self.discover = discover
        self.result = {}
        if headless == True:
            self.options.add_argument('--headless')
            self.options.add_argument('--disable-gpu')
            
    def run(self):
        self._crawl_main()
        self._crawl_detail()
    
    def _crawl_main(self):
        if platform.system() == 'Windows':
            driver = webdriver.Chrome("C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe",
                                                  options=self.options)

        if platform.system() == 'Darwin':
            driver = webdriver.Chrome("/Users/Lunar/chromedriver/chromedriver",
                                                  options=self.options)
        
        print('Crawling main...')
        driver.get('https://movie.douban.com/tv/')
        time.sleep(1)
        
        driver.find_element_by_xpath("//input[@value='国产剧']/..").click()
        time.sleep(1)
        
        for i in range(self.scrolls):
            try:
                if driver.find_element_by_xpath("//a[text()='加载更多']").is_displayed():
                    driver.find_element_by_xpath("//a[text()='加载更多']").click()
                    time.sleep(1+random.random())
            except:
                time.sleep(3)
            print('{} / {}'.format(i+1, self.scrolls), end='\r')
            
        f = etree.HTML(driver.page_source)
        self.ids += f.xpath('//div[@class=\'cover-wp\']/@data-id')
            
        print(len(self.ids), self.ids[:5])
        print('Main crawled')
        
    def _generate_tv_pages(self, id_):
        head = 'https://movie.douban.com/subject/'
        tail = random.choice(['/?tag=国产剧&from=gaia_video', '/?from=subject-page'])
        return head + str(id_) + tail
    
    def _crawl_detail(self):
        print('Crawling details...')
        if self.discover:
            iters = self.discover
        else:
            iters = len(self.ids)
        for t in tqdm(range(iters)):
            if t % 100 == 0:
                headers = {'user-agent': random.choice(self.user_agent)}
            try:
                if len(self.ids) > 0:
                    id_ = random.choice(self.ids)
                    url = self._generate_tv_pages(id_)
                    text = requests.get(url=url, headers=headers).text
                    if self.discover:
                        self._discover_detail(text)
                    self._parse_details(text)
                    self.ids.remove(id_)
                    time.sleep(self.sleep + random.random() * self.sleep)
                    
                    if (t+1 + random.randint(0, 5)) % 100 == 0:
                        time.sleep(self.inter_sleep + random.random() * self.inter_sleep)
                        
                else: break
            except: pass
        print('Done')
    
    # discvoer url from detail page and update the ids to crawl
    def _discover_detail(self, text):
        f = etree.HTML(text)
        found = f.xpath("//div[@id='recommendations']//dt/a[@data-moreurl-dict]/@data-moreurl-dict")
        found =  [eval(i)['subject_id'] for i in found]
        found = [i for i in found if i not in self.result]
        self.ids += found
        self.ids = list(set(self.ids))
        
    def _parse_details(self, text):
        f = etree.HTML(text)
        content = json.loads(f.xpath("//script[@type='application/ld+json']")[0].text)
        
        id_ = content['url'].split('/')[-2]
        parsed = {}
        parsed['name'] = content['name']
        parsed['date_published'] = content['datePublished']
        parsed['director'] = {i['url'].split('/')[-2]: i['name'].split(' ')[0] for i in content['director']}
        parsed['author'] = {i['url'].split('/')[-2]: i['name'].split(' ')[0] for i in content['author']}

        hold =[(i['url'].split('/')[-2], i['name'].split(' ')[0]) for i in content['actor']]
        parsed['main_actor'] = {id_: name for id_, name in hold[:8]}
        parsed['support_actor'] = {id_: name for id_, name in hold[8:]}

        parsed['genre'] = content['genre']
        parsed['duration'] = content['duration']
        parsed['rating'] = content['aggregateRating']['ratingValue']
        parsed['rating_count'] = content['aggregateRating']['ratingCount']
        parsed['rating_best'] = content['aggregateRating']['bestRating']
        parsed['rating_worst'] = content['aggregateRating']['worstRating']

        parsed['location'] = f.xpath("//div[@id='info']//span[text()='制片国家/地区:']")[0].tail.strip()
        parsed['num_episode'] = f.xpath("//div[@id='info']//span[text()='集数:']")[0].tail.strip()
                
        self.result[id_] = parsed



def clean(dic):
    id_to_pop = []
    for i in dic:
        if (len(dic[i]['author']) == 0 or len(dic[i]['director']) == 0) or ((
            len(dic[i]['main_actor']) == 0 or len(dic[i]['support_actor']) == 0) or 
            dic[i]['location'] != '中国大陆'):
            id_to_pop.append(i)
    for i in id_to_pop:
        dic.pop(i)
    return dic

def get_true_score(dic, typ='main_actor'):
    char_dic= {}
    for i in dic:
        char_dic = dict(char_dic, **dic[i][typ])
    char_list = list(char_dic.keys())
    tv_list = list(dic.keys())
    rating_list = [float(dic[i]['rating']) for i in tv_list]
    df = pd.DataFrame(index = tv_list, columns=char_list).fillna(0)
    for i in tqdm(tv_list):
        for j in char_list:
            if j in dic[i][typ]:
                df.loc[i, j] = 1
    count = df.sum(1).values
    df = df / count[:, None]
    scores = np.linalg.pinv(df.values) @ np.array(rating_list).reshape(-1, 1)
    scores = {i: j for (i, j) in zip(char_list, scores)}
    return scores

def get_avg_score(dic, typ='main_actor'):
    char_dic= {}
    for i in dic:
        char_dic = dict(char_dic, **dic[i][typ])
    char_list = list(char_dic.keys())
    tv_list = list(dic.keys())
    rating_list = [float(dic[i]['rating']) for i in tv_list]
    df = pd.DataFrame(index = char_list, columns=tv_list)
    for i in tqdm(tv_list):
        for j in char_list:
            if j in dic[i][typ]:
                df.loc[j, i] = float(dic[i]['rating'])
    df['score'] = df.mean(1)
    scores = {i: j for (i, j) in zip(char_list, df['score'])}
    count = df.notnull().sum(1)
    count = {i: j for (i, j) in zip(char_list, count)}
    return scores, count

def get_mapping(dic):
    char_dic= {}
    for i in dic:
        char_dic = dict(char_dic, **dic[i]['author'])
        char_dic = dict(char_dic, **dic[i]['director'])
        char_dic = dict(char_dic, **dic[i]['main_actor'])
        char_dic = dict(char_dic, **dic[i]['support_actor'])
    return char_dic

In [9]:
crawl = CrawlDoubanTV(scrolls=25, discover=2000, sleep=0.8, inter_sleep=5)
crawl.run()

Crawling main...
500 ['27195020', '26794191', '26660368', '33413766', '30191989']
Main crawled
Crawling details...



Done


In [10]:
mapping = get_mapping(crawl.result)
print('Num of shows crawed: %s'%len(crawl.result))
print('Num of chars crawed: %s'%len(mapping))

dic = crawl.result.copy()
with open('save.pickle', 'wb') as handle:
    pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)
dic = clean(dic)

Num of shows crawed: 442
Num of chars crawed: 6630


In [58]:
for i in dic:
    dic[i]['actor'] = dict(dic[i]['main_actor'], **dic[i]['support_actor'] )
    
main_avg, main_count = get_avg_score(dic, typ='main_actor')
support_avg, support_count = get_avg_score(dic, typ='support_actor')
true = get_true_score(dic, typ='actor')

df = pd.DataFrame(main_avg, index=['main_avg']).T.join(
       pd.DataFrame(support_avg, index=['support_avg']).T, how='outer').join(
       pd.DataFrame(true, index=['true_score']).T, how='outer').join(
       pd.DataFrame(main_count, index=['main_count']).T, how='outer').join(
       pd.DataFrame(support_count, index=['support_count']).T, how='outer')

df = df.fillna(0)

df.loc[df['support_avg'] == 0, 'support_avg'] = df.loc[df['support_avg'] == 0, 'main_avg']
df['avg_score'] = np.round(df['main_avg'] * 0.6 + df['support_avg'] * 0.4, 2)
df.loc[df['main_avg'] == 0, 'avg_score'] = df.loc[df['main_avg'] == 0, 'support_avg'] * 0.8

df['true_score'] = df['avg_score'].min() + (df['true_score'] - df['true_score'].min() ) / (
    df['true_score'].max() -  df['true_score'].min()) * (10 - df['avg_score'].min())

df['total'] = df['main_count'] + df['support_count']
df['main_pct'] = df['main_count'] / df['total']
df['diff'] = df['true_score'] - df['avg_score']
        
df = df.drop(['main_avg', 'support_avg', 'support_count'], axis=1)
df = df.rename(index=mapping)

In [137]:
df = df[df['total'] >= 5]
df.columns

Index(['true_score', 'main_count', 'avg_score', 'total', 'main_pct', 'diff'], dtype='object')

In [138]:
# Good performance
df[df['main_count'] >= 5].sort_values(by='true_score', ascending=False).head(10)

,true_score,main_count,avg_score,total,main_pct,diff
王劲松,10.000000,6.0,7.10,15.0,0.400000,2.900000
孙俪,9.569861,11.0,7.98,11.0,1.000000,1.589861
吕中,9.378229,7.0,8.58,13.0,0.538462,0.798229
李光复,9.290976,6.0,7.73,11.0,0.545455,1.560976
阿杰,9.138173,8.0,7.49,32.0,0.250000,1.648173
陈道明,9.083896,8.0,7.64,10.0,0.800000,1.443896
倪大红,8.621444,12.0,7.89,20.0,0.600000,0.731444
刘奕君,8.195573,9.0,7.39,16.0,0.562500,0.805573
罗海琼,8.147200,5.0,7.76,9.0,0.555556,0.387200
郭京飞,8.133652,7.0,7.83,10.0,0.700000,0.303652


In [139]:
# Bad performance
df[df['main_count'] >= 5].sort_values(by='true_score', ascending=True).head(10)

,true_score,main_count,avg_score,total,main_pct,diff
李易峰,2.578834,5.0,5.38,5.0,1.000000,-2.801166
冯绍峰,2.935735,5.0,6.16,8.0,0.625000,-3.224265
唐嫣,3.140345,10.0,5.68,10.0,1.000000,-2.539655
陈伟霆,3.151760,6.0,5.74,6.0,1.000000,-2.588240
邓伦,3.183926,7.0,6.62,9.0,0.777778,-3.436074
张智尧,3.395259,5.0,5.62,5.0,1.000000,-2.224741
魏千翔,3.470649,5.0,5.31,8.0,0.625000,-1.839351
祖峰,3.529117,6.0,7.13,9.0,0.666667,-3.600883
辛芷蕾,4.004925,5.0,5.32,5.0,1.000000,-1.315075
王子文,4.078649,5.0,6.93,5.0,1.000000,-2.851351


In [140]:
# Key figue and good performance
df[(df['main_count'] >= 5) & (df['main_pct'] >= 0.8)].sort_values(by='true_score', ascending=False).head(10)

,true_score,main_count,avg_score,total,main_pct,diff
孙俪,9.569861,11.0,7.98,11.0,1.0,1.589861
陈道明,9.083896,8.0,7.64,10.0,0.8,1.443896
张嘉译,8.023180,13.0,7.53,13.0,1.0,0.493180
胡歌,7.713247,12.0,7.42,15.0,0.8,0.293247
吴刚,6.998385,5.0,8.30,5.0,1.0,-1.301615
张一山,6.587124,7.0,7.75,7.0,1.0,-1.162876
藤新,6.563231,6.0,8.70,6.0,1.0,-2.136769
吴秀波,6.553886,7.0,8.15,7.0,1.0,-1.596114
王传君,6.504797,5.0,7.57,5.0,1.0,-1.065203
孙艺洲,6.504797,5.0,7.57,5.0,1.0,-1.065203


In [141]:
# Key figue and bad performance
df[(df['main_count'] >= 5) & (df['main_pct'] >= 0.8)].sort_values(by='true_score', ascending=True).head(10)

,true_score,main_count,avg_score,total,main_pct,diff
李易峰,2.578834,5.0,5.38,5.0,1.0,-2.801166
唐嫣,3.140345,10.0,5.68,10.0,1.0,-2.539655
陈伟霆,3.151760,6.0,5.74,6.0,1.0,-2.588240
张智尧,3.395259,5.0,5.62,5.0,1.0,-2.224741
辛芷蕾,4.004925,5.0,5.32,5.0,1.0,-1.315075
王子文,4.078649,5.0,6.93,5.0,1.0,-2.851351
曾舜晞,4.162308,5.0,5.18,5.0,1.0,-1.017692
袁立,4.249949,5.0,7.85,5.0,1.0,-3.600051
钟汉良,4.435712,5.0,8.15,5.0,1.0,-3.714288
白冰,4.559800,6.0,5.82,6.0,1.0,-1.260200


In [142]:
# Small figue and good performance
df[(df['total'] >= 10) & (df['main_pct'] <= 0.2)].sort_values(by='true_score', ascending=False).head(10)

,true_score,main_count,avg_score,total,main_pct,diff
陆建艺,8.947914,0.0,6.305455,12.0,0.000000,2.642459
戴春荣,8.541161,2.0,8.160000,13.0,0.153846,0.381161
沈保平,8.107954,2.0,8.040000,15.0,0.133333,0.067954
秦焰,7.867371,2.0,8.580000,13.0,0.153846,-0.712629
王永泉,7.824675,2.0,8.050000,18.0,0.111111,-0.225325
杜玉明,7.669468,2.0,7.580000,10.0,0.200000,0.089468
邓莎,7.667066,2.0,8.280000,11.0,0.181818,-0.612934
姜广涛,7.660134,3.0,8.020000,27.0,0.111111,-0.359866
边江,7.025425,3.0,8.090000,16.0,0.187500,-1.064575
王宏,6.913633,0.0,6.589091,12.0,0.000000,0.324542


In [143]:
# Small figue and bad performance
df[(df['total'] >= 10) & (df['main_pct'] <= 0.2)].sort_values(by='true_score', ascending=True).head(10)

,true_score,main_count,avg_score,total,main_pct,diff
冯晖,4.692406,2.0,7.650000,12.0,0.166667,-2.957594
岳旸,4.741778,2.0,7.920000,13.0,0.153846,-3.178222
张陆,4.895472,2.0,7.280000,10.0,0.200000,-2.384528
刘校妤,5.322950,2.0,7.570000,11.0,0.181818,-2.247050
宗峰岩,5.569979,0.0,5.395556,10.0,0.000000,0.174424
曹卫宇,6.096334,2.0,7.600000,11.0,0.181818,-1.503666
邓小鸥,6.338803,0.0,6.189091,12.0,0.000000,0.149712
宁文彤,6.588101,0.0,6.044444,10.0,0.000000,0.543656
王仁君,6.677234,2.0,6.020000,10.0,0.200000,0.657234
张晓谦,6.688536,2.0,7.410000,10.0,0.200000,-0.721464


In [153]:
# Hign avg_score but low performance
df[((df['avg_score'] >= 7) & (df['true_score'] <= 5)) & (df['main_count'] >= 4)].sort_values(by='avg_score', ascending=False).head(10)

,true_score,main_count,avg_score,total,main_pct,diff
赵达,4.843516,4.0,8.47,8.0,0.500000,-3.626484
钟汉良,4.435712,5.0,8.15,5.0,1.000000,-3.714288
袁立,4.249949,5.0,7.85,5.0,1.000000,-3.600051
归亚蕾,4.731149,5.0,7.83,5.0,1.000000,-3.098851
马伊琍,3.914696,4.0,7.82,6.0,0.666667,-3.905304
蒋勤勤,4.727689,5.0,7.52,5.0,1.000000,-2.792311
董勇,4.776336,5.0,7.47,8.0,0.625000,-2.693664
陈龙,4.595431,5.0,7.30,9.0,0.555556,-2.704569
高露,4.953980,4.0,7.26,6.0,0.666667,-2.306020
寇振海,4.776459,4.0,7.20,6.0,0.666667,-2.423541


In [154]:
# Hign performance but low avg_score
df[((df['avg_score'] <= 5) & (df['true_score'] >= 7)) & (df['main_count'] >= 0)].sort_values(by='avg_score', ascending=False).head(10)

,true_score,main_count,avg_score,total,main_pct,diff


In [163]:
df.loc[['王俊凯', '王源', '易烊千玺', '朱一龙']]

,true_score,main_count,avg_score,total,main_pct,diff
王俊凯,5.624300,4.0,6.74,6.0,0.666667,-1.115700
王源,4.578812,3.0,6.27,5.0,0.600000,-1.691188
易烊千玺,5.630033,4.0,6.94,6.0,0.666667,-1.309967
朱一龙,4.340572,5.0,6.31,7.0,0.714286,-1.969428


In [170]:
text = '雷佳音 / 易烊千玺 / 周一围 / 芦芳生 / 热依扎 / 王鹤润 / 杰曼·翰苏 / 韩童生 / 李媛 / 余皑磊 / 艾如 / 蔡鹭 / 赵魏 / 冯嘉怡 / 吕凉 / 吴晓亮 / 徐璐 / 高叶 / 王思思 / 曲栅栅'
df.loc[text.split(' / ')]['true_score'].mean()

5.705814076934514

5.705814076934514